In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
iris = datasets.load_iris()

In [3]:
# Выделим два класса для нашей модели
iris.target_names[1:]

array(['versicolor', 'virginica'], dtype='<U10')

In [4]:
# Выделим значения двух классов
X = iris.data[50:]
y = iris.target[50:]

In [5]:
# Значениям классов присвоим 0 и 1
y = y - 1

In [6]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [7]:
# В матрицу значений признаков добавим фиктивный признак (свободный член модели логистической регрессии)
X = np.c_[np.ones((100,1)),X]

In [8]:
X[:5]

array([[1. , 7. , 3.2, 4.7, 1.4],
       [1. , 6.4, 3.2, 4.5, 1.5],
       [1. , 6.9, 3.1, 4.9, 1.5],
       [1. , 5.5, 2.3, 4. , 1.3],
       [1. , 6.5, 2.8, 4.6, 1.5]])

Модель логистической регрессии: 
$L_\theta = \theta_0 + \theta_1X_1 + \theta_2X_2 + \theta_3X_3 + \theta_4X_4$  
  
  В матричном виде: $L = X\theta$,  где $X = {\begin{pmatrix} 1 & x_{11} & ... & x_{14}\\  1 & x_{21} & ... & x_{24}\\ .&.&.&.\\1 & x_{41} & ... & x_{44}\end{pmatrix}}$,  $\theta = {\begin{pmatrix}\theta_0 \\ \theta_1 \\ \theta_2 \\ \theta_3 \\ \theta_4\end{pmatrix}}$
  
Также можно записать в виде: 
$$L(x_i) = \theta_0 + \theta_1x_{i1} + \theta_2x_{i2} + \theta_3x_{i3} + \theta_4x_{i4}$$  
Вероятности вычисляются по формуле: $$h_\theta(x_i) = \frac{1}{1 + e^{-L_\theta(x_i)}}$$  
Для двух классов функция ошибки имеет вид:  
$$J = -\sum_{i=1}^{100}\big(y_i\ln(h_\theta(x_i)) + (1 - y_i)\ln(h_\theta(x_i))\big)$$  
$y_i$ принимают значения: 0 или 1.  
  
Подставив выражение для $h_\theta(x_i)$ получим:  
$$J = \sum_{i=1}^{100}\big(\ln(1 + e^{-L(x_i)}) + (1 - y_i)L(x_i)\big)$$
  
Градиент функции ошибки от параметра $\theta$ - это частные производные по значениям параметра $\theta_j$:  

$$\frac{\partial J(\theta)}{\partial \theta_j} = \sum_{i=1}^{100}\big(1 - y_i + \frac{e^{-L(x_i)}}{1 + e^{-L(x_i)}}\big)\frac{\partial L(x_i)}{\partial \theta_j}$$  

В векторной форме:  
$$grad = {\nabla_{\theta}J(\theta)} = {\begin{pmatrix}J'_{\theta_0}\\J'_{\theta_1}\\J'_{\theta_2}\\J'_{\theta_3}\\J'_{\theta_4} \end{pmatrix}} = {\begin{pmatrix}\frac{1}{1 + e^{-X\theta}} - y\end{pmatrix}}X$$



---
##   <p style="text-align: center;">Метод градиентного спуска</p>


$$\Delta\theta = -\eta\nabla_{\theta}J(\theta)$$  

$$\theta = \theta + \Delta\theta = \theta - \eta\nabla_{\theta}J(\theta)$$

In [9]:
def gradient_log_reg(eta, X, y, theta, iterations):
    
    """Функция реализует метод градиентного спуска для логистической регрессии"""
    
    cost_gd = np.array([])
    theta_gd = []
    
    for i in range(iterations):
                      
        gradient = np.dot(1 / (1 + np.exp(- np.dot(X, theta))) - y, X)
        
        theta = theta - eta * gradient
        
        J = np.sum((1 - y)*np.dot(X, theta) + np.log(1 + np.exp(- np.dot(X, theta)))) # Loss function for 
                                                                                      # Logistic Regression
                      
        cost_gd = np.append(cost_gd, J)
        theta_gd.append(theta.T)
                     
    return cost_gd, theta_gd

In [10]:
def prediction_score(theta):
    
    """По параметру theta вычисляет вероятности для значений признаков, округлением присваивает класс (если 
    вероятность больше 0.5, то присваивает класс 1, в противном случае класс 0) и выводит количество совпавших  
    с целевым признаком классов, делённое на 100."""
    
    pred = np.zeros(100)
    for i in range(100):
        pred[i] = np.round(1 / (1 + np.exp(- np.dot(X[i], theta))))
    return 1 - sum(np.abs(y - pred)) / 100

In [11]:
theta0 = np.array([0.1, 0.1, 0.1, 0.1, 0.1])

In [12]:
cost_gd, theta_gd = gradient_log_reg(0.01, X, y, theta0, 40000)

In [13]:
cost_gd[-10:]

array([6.07585666, 6.07585108, 6.0758455 , 6.07583992, 6.07583434,
       6.07582876, 6.07582318, 6.0758176 , 6.07581202, 6.07580644])

In [14]:
theta_gd[-1]

array([-31.1008369 ,  -2.70947193,  -5.74402572,   7.98847701,
        14.96589469])

In [15]:
prediction_score(theta_gd[-1])

0.98

На 40000-й итерации для параметров: $\theta_0=(0.1,\ \ 0.1,\ \ 0.1,\ \ 0.1)$ и $\eta=0.01$ значение функции потери $J=6.07580644$.

---
## <p style="text-align: center;">Ускоренные градиенты Нестерова (NAG)</p>

  

$$\Delta_{p+1} = \gamma\Delta_p + \eta\nabla{J(\theta_p - \gamma\Delta_p)}$$  

$$\theta_{p+1} = \theta_p - \Delta_{p+1}$$

In [16]:
def NAG(eta, gamma, X, y, theta, iterations):
    
    """Функция реализует метод ускоренных градиентов Нестерова для логистической регрессии"""
    
    delta = np.zeros(5)
    cost_nag = np.array([])
    theta_nag = []
    
    for i in range(iterations):
        
        gradient = np.dot(1 / (1 + np.exp(- np.dot(X, theta - gamma * delta))) - y, X)
        
        delta = gamma * delta + eta * gradient
        theta = theta - delta
        
        J = np.sum((1 - y)*np.dot(X, theta) + np.log(1 + np.exp(- np.dot(X, theta)))) # Loss function for 
                                                                                      # Logistic Regression
        
        cost_nag = np.append(cost_nag, J)
        theta_nag.append(theta.T)
        
    return cost_nag, np.array(theta_nag)

In [17]:
cost_nag, theta_nag = NAG(0.01, 0.9, X, y, theta0, 40000)

In [18]:
# Значения функции потери при последних 5 итераций
cost_nag[-5:]

array([5.94927403, 5.94927403, 5.94927403, 5.94927403, 5.94927403])

In [19]:
# Значения θ, полученные при последней итерации
theta_nag[-1]

array([-42.60889494,  -2.4655599 ,  -6.67835358,   9.42539695,
        18.27758572])

In [20]:
# Предсказание для последнего значения θ
prediction_score(theta_nag[-1])

0.98

При оптимизации методом Нестерова сходимость быстрее. При увеличении параметра $\gamma$ на 0.05 сходимость ускоряется к значению $J\approx5.9492$.

---
## <p style="text-align: center;">Метод RMSprop</p>

$$\nabla{J(\theta_p)} = {\begin{pmatrix}g_p^{(1)}, & g_p^{(2)}, & g_p^{(3)}, & g_p^{(4)}\end{pmatrix}}$$  
  
  Экспоненциально взвешенное скользящее среднее градиентов:  
  
$$EG_{p+1}^{(i)} = \gamma{EG_p^{(i)}} + (1 - \gamma)\big(g_p^{(i)}\big)^2$$  
$$\theta_{p+1} = \theta_p - \frac{\eta}{\sqrt{EG_{p+1}+\epsilon}}\nabla{J(\theta_p)},\           \ где\     \ \epsilon\approx 10^{-8}$$

In [21]:
def RMSprop(eta, gamma, X, y, theta, iterations):
    
    """Функция реализует метод RMSprop для логистической регрессии"""
    
    eps = 0.00000001
    EG = (np.dot(1 / (1 + np.exp(- np.dot(X, theta))) - y, X))**2
    cost_rms = np.array([])
    theta_rms = []
    
    for i in range(iterations):
        
        gradient = np.dot(1 / (1 + np.exp(- np.dot(X, theta))) - y, X)
        
        EG = gamma * EG + (1 - gamma) * (gradient**2)
        
        theta = theta - (eta / np.sqrt(EG + eps)) * gradient
                
        J = np.sum((1 - y)*np.dot(X, theta) + np.log(1 + np.exp(- np.dot(X, theta)))) # Loss function for 
                                                                                      # Logistic Regression
        
        cost_rms = np.append(cost_rms, J)
        theta_rms.append(theta.T)
        
    return cost_rms, np.array(theta_rms)

In [22]:
cost_rms, theta_rms = RMSprop(0.01, 0.9, X, y, theta0, 40000)

In [23]:
# Значения функции потери при последних 5 итерациях
cost_rms[-5:]

array([5.95630284, 5.9562139 , 5.95630259, 5.95621365, 5.95630234])

In [24]:
# Значения θ, полученные при последней итерации
theta_rms[-1]

array([-41.66807294,  -2.47175781,  -6.58652312,   9.30024223,
        18.00176945])

In [25]:
# Предсказание для последнего значения θ
prediction_score(theta_rms[-1])

0.98

Для метода **RMSprop** сходимость к минимуму функции потери немного медленне, чем при методе Нестерова.